In [1]:
import os
import psycopg2
from openai import OpenAI
import json
import time
import re

# Configure OpenAI API
llm = OpenAI(
    api_key="sk-16a90ba86cfc4dcf9402bea1309c9021",  # Ensure to secure your API key
    base_url="https://api.deepseek.com"
)

# PostgreSQL Database Connection Parameters
DATABASE = {
    'database': os.getenv('DB_NAME', 'maude'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', '12345687'),
    'host': os.getenv('DB_HOST', '192.168.8.167'),
    'port': os.getenv('DB_PORT', '5432')
}
SCHEMA = 'maude'

def generate_response(prompt):
    """
    Generate a response from OpenAI based on the provided prompt.
    """
    try:
        response = llm.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": prompt},
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None

def read_prompt_file(file_path):
    """
    Read the content of the prompt file.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

def connect_database():
    """
    Establish a connection to the PostgreSQL database.
    """
    try:
        conn = psycopg2.connect(**DATABASE)
        cursor = conn.cursor()
        print("Successfully connected to the database.")
        return conn, cursor
    except Exception as e:
        print(f"Database connection failed: {e}")
        return None, None

def get_table_structure(cursor, table_name):
    """
    Retrieve the structure of a specified table.
    """
    try:
        query = f"""
        SELECT 
            column_name, 
            data_type, 
            character_maximum_length, 
            is_nullable
        FROM 
            information_schema.columns
        WHERE 
            table_schema = '{SCHEMA}' 
            AND table_name = '{table_name}';
        """
        cursor.execute(query)
        columns = cursor.fetchall()
        structure = []
        for col in columns:
            structure.append({
                'column_name': col[0],
                'data_type': col[1],
                'character_max_length': col[2],
                'is_nullable': col[3]
            })
        return structure
    except Exception as e:
        print(f"Error retrieving table structure ({table_name}): {e}")
        return None

def get_sample_data(cursor, table_name, limit=3):
    """
    Retrieve sample data from a specified table.
    """
    try:
        query = f'SELECT * FROM {SCHEMA}."{table_name}" LIMIT {limit};'
        cursor.execute(query)
        rows = cursor.fetchall()
        # Retrieve column names
        col_names = [desc[0] for desc in cursor.description]
        sample_data = [dict(zip(col_names, row)) for row in rows]
        return sample_data
    except Exception as e:
        print(f"Error retrieving sample data ({table_name}): {e}")
        return None

def execute_sql(cursor, sql_query):
    """
    Execute an SQL query and handle the results.
    """
    try:
        cursor.execute(sql_query)
        # If it's a SELECT query, fetch and return the results
        if sql_query.strip().upper().startswith("SELECT"):
            rows = cursor.fetchall()
            col_names = [desc[0] for desc in cursor.description]
            data = [dict(zip(col_names, row)) for row in rows]
            return data, None
        else:
            # For non-SELECT queries, commit the transaction
            cursor.connection.commit()
            return None, None
    except Exception as e:
        return None, str(e)

def extract_table_names(prompt_content):
    """
    Extract real table names from the prompt content.
    Assumes that table names are mentioned in the format `merged table xxx` and replaces them with actual table names.
    For demonstration, uses a predefined mapping.
    """
    # Define a mapping from merged table names to actual table names
    # This should be updated based on actual mappings
    merged_to_real = {
        "Merged_Table_1": "mdrfoi",
        "Merged_Table_2": "patientproblemcode",
        "Merged_Table_3": "some_other_table",  # Replace with actual table names
        # Add all necessary mappings here
    }
    
    # Extract merged table names using regex
    merged_tables = re.findall(r'Merged_Table_\d+', prompt_content)
    
    # Replace merged table names with real table names
    involved_tables = []
    for mt in merged_tables:
        real_table = merged_to_real.get(mt, None)
        if real_table:
            involved_tables.append(real_table)
        else:
            print(f"No real table mapping found for {mt}. Please update the mapping.")
    
    # If no merged tables found, use default involved tables
    if not involved_tables:
        involved_tables = ["mdrfoi", "DEVICE", "deviceproblemcodes", "patient", "patientproblemcode", "foitext", "DISCLAIM"]
    
    return involved_tables

def analyze_data(research_question, data):
    """
    Analyze the data to validate the research question.
    This function can be expanded based on specific analysis requirements.
    For demonstration, it sends the data and research question to OpenAI for analysis.
    """
    if not data:
        print("No data available for analysis.")
        return
    
    # Convert data to JSON string
    data_json = json.dumps(data, ensure_ascii=False)
    
    # Create analysis prompt
    analysis_prompt = (
        f"Based on the following research question and data, analyze the validity and feasibility of the research question.\n\n"
        f"Research Question: {research_question}\n\n"
        f"Data: {data_json}\n\n"
        f"Provide a detailed analysis report:"
    )
    
    # Get analysis report from OpenAI
    analysis_report = generate_response(analysis_prompt)
    if analysis_report:
        print(f"Analysis Report:\n{analysis_report}")
    else:
        print("Failed to generate analysis report.")



In [2]:
# 1. Read table information from prompt.txt
prompt_file = 'prompt.txt'
prompt_content = read_prompt_file(prompt_file)
if not prompt_content:
    print("Failed to read prompt.txt file.")

# 2. Generate a research question
research_prompt = (
    f"Based on the following MAUDE database table information, propose a meaningful research question.\n\n"
    f"{prompt_content}"
)
research_question = generate_response(research_prompt)
if not research_question:
    print("Failed to generate a research question.")
print(f"Proposed Research Question:\n\n{research_question}\n")

Proposed Research Question:

### Research Question:

**How do the rates of adverse events and device problems vary across different types of medical devices, and what factors (such as device age, manufacturer, and patient demographics) are associated with these variations?**

### Rationale:

This research question is designed to leverage the rich dataset provided by the MAUDE database to explore the following key aspects:

1. **Variation in Adverse Events and Device Problems:**
   - By analyzing the `Event Type`, `Device Problem`, and `Patient Problem` fields, we can identify which types of medical devices are most prone to adverse events and device malfunctions.
   - This analysis can help in understanding the relative safety and reliability of different device categories.

2. **Device Age and Adverse Events:**
   - The `Device Age` field can be used to investigate whether the age of the device (e.g., newer vs. older devices) is correlated with the occurrence of adverse events.
   - T

In [3]:
# 3. Plan execution steps based on the research question
planning_prompt = (
    f"Based on the following research question, outline specific execution steps, including which tables and fields need to be queried.\n\n"
    f"Research Question: {research_question}"
)
execution_steps = generate_response(planning_prompt)
if not execution_steps:
    print("Failed to plan execution steps.")
print(f"Planned Execution Steps:\n\n{execution_steps}\n")

Planned Execution Steps:

To address the research question, we need to follow a structured approach that involves querying specific tables and fields from the MAUDE (Manufacturer and User Facility Device Experience) database. Below is a detailed outline of the execution steps, including the tables and fields that need to be queried.

### 1. Data Cleaning and Preprocessing

**Steps:**
- **Identify Relevant Tables:**
  - `DeviceTable`: Contains information about the devices.
  - `EventTable`: Contains information about adverse events and device problems.
  - `PatientTable`: Contains demographic information about the patients.
  - `ManufacturerTable`: Contains information about the manufacturers.
  - `LocationTable`: Contains information about the event locations.

- **Merge Relevant Tables:**
  - Create a comprehensive dataset by merging the above tables on common keys (e.g., `DeviceID`, `EventID`, `PatientID`, `ManufacturerID`).

- **Handle Missing Values:**
  - Identify and handle miss

In [4]:
# 4. Connect to the database
conn, cursor = connect_database()
if not conn or not cursor:
    print("database connection failed")

# 5. Identify involved tables by extracting from execution steps
involved_tables = extract_table_names(execution_steps)
print(f"Identified Involved Tables: {involved_tables}\n")

Successfully connected to the database.
Identified Involved Tables: ['mdrfoi', 'DEVICE', 'deviceproblemcodes', 'patient', 'patientproblemcode', 'foitext', 'DISCLAIM']



In [5]:
# 6. Check table structures and sample data
table_info = {}
for table in involved_tables:
    # Check if table exists
    structure = get_table_structure(cursor, table)
    if structure is None or not structure:
        print(f"Table '{table}' does not exist or has no columns. Skipping.")
    samples = get_sample_data(cursor, table)
    if samples is None:
        print(f"Unable to retrieve sample data for table: {table}")
    table_info[table] = {
        'structure': structure,
        'samples': samples
    }
    print(f"Table: {table}")
    print(f"Structure: {json.dumps(structure, ensure_ascii=False, indent=2)}")
    print(f"Sample Data: {json.dumps(samples, ensure_ascii=False, indent=2)}\n")

if not table_info:
    print("No valid tables found for analysis. Exiting.")
    cursor.close()
    conn.close()

# Convert table information to JSON for OpenAI processing
table_info_json = json.dumps(table_info, ensure_ascii=False)

Table: mdrfoi
Structure: [
  {
    "column_name": "mdr_report_key",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "event_key",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "report_number",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "report_source_code",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "manufacturer_link_flag",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "number_devices_in_event",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "number_patients_in_event",
    "data_type": "text",
    "character_max_length": null,
    "is_nullable": "YES"
  },
  {
    "column_name": "date_received"

In [6]:
sql_prompt = (
    f"Based on the following execution steps and table information, generate SQL queries to answer the research question.\n\n"
    f"Research Question: {research_question}\n\n"
    f"Execution Steps: {execution_steps}\n\n"
    f"Table Information: {table_info_json}\n\n"
    f"Ensure that table names are formatted as maude.\"TABLENAME\" and use the correct column names as per the table structures.\n\n"
    f"**Constraint**: Please ensure that the SQL query does not return more than 10 hits by limited 10.\n\n"
    f"**Additional Requirement**: Each generated SQL statement should be self-contained and independent, meaning they should not rely on the execution of other SQL statements. If there are dependencies between queries, combine them into a single, cohesive SQL statement."
)
sql_query_full = generate_response(sql_prompt)
if not sql_query_full:
    print("Failed to generate SQL query.")
    cursor.close()
    conn.close()
print(f"Generated SQL Query:\n\n{sql_query_full}\n")


Generated SQL Query:

To address the research question and follow the execution steps, we need to generate SQL queries that cover the following aspects:

1. **Variation in Adverse Events and Device Problems by Device Type**
2. **Device Age and Adverse Events**
3. **Manufacturer Influence**
4. **Patient Demographics**
5. **Event Location and Reporting Patterns**

Given the constraints and requirements, here are the SQL queries:

### 1. Variation in Adverse Events and Device Problems by Device Type

```sql
SELECT 
    maude."DEVICE".generic_name AS DeviceType,
    COUNT(maude."mdrfoi".mdr_report_key) AS EventCount
FROM 
    maude."mdrfoi"
JOIN 
    maude."DEVICE" ON maude."mdrfoi".mdr_report_key = maude."DEVICE".mdr_report_key
WHERE 
    maude."mdrfoi".adverse_event_flag = 'Y'
GROUP BY 
    maude."DEVICE".generic_name
ORDER BY 
    EventCount DESC
LIMIT 10;
```

### 2. Device Age and Adverse Events

```sql
SELECT 
    maude."DEVICE".device_age_text AS DeviceAge,
    COUNT(maude."mdrfoi".

In [7]:
import re
pattern = r'```sql\n(.*?)```'
matches = re.findall(pattern, sql_query_full, re.DOTALL)
if matches: # Extract the SQL query and remove any leading/trailing whitespace 
    sql_queries = [match.strip() for match in matches]
    print("Extracted SQL Query:\n")
    # 遍历 sql_queries 数组并逐一打印每个 SQL 查询
    for idx, query in enumerate(sql_queries, start=1):
        print(f"--- SQL Query {idx} ---")
        print(query)
        print("\n")  # 添加空行以增加可读性
else: 
    print("No SQL code block found in sql_query_full.")


Extracted SQL Query:

--- SQL Query 1 ---
SELECT 
    maude."DEVICE".generic_name AS DeviceType,
    COUNT(maude."mdrfoi".mdr_report_key) AS EventCount
FROM 
    maude."mdrfoi"
JOIN 
    maude."DEVICE" ON maude."mdrfoi".mdr_report_key = maude."DEVICE".mdr_report_key
WHERE 
    maude."mdrfoi".adverse_event_flag = 'Y'
GROUP BY 
    maude."DEVICE".generic_name
ORDER BY 
    EventCount DESC
LIMIT 10;


--- SQL Query 2 ---
SELECT 
    maude."DEVICE".device_age_text AS DeviceAge,
    COUNT(maude."mdrfoi".mdr_report_key) AS EventCount
FROM 
    maude."mdrfoi"
JOIN 
    maude."DEVICE" ON maude."mdrfoi".mdr_report_key = maude."DEVICE".mdr_report_key
WHERE 
    maude."mdrfoi".adverse_event_flag = 'Y'
GROUP BY 
    maude."DEVICE".device_age_text
ORDER BY 
    EventCount DESC
LIMIT 10;


--- SQL Query 3 ---
SELECT 
    maude."DEVICE".manufacturer_d_name AS ManufacturerName,
    COUNT(maude."mdrfoi".mdr_report_key) AS EventCount
FROM 
    maude."mdrfoi"
JOIN 
    maude."DEVICE" ON maude."mdrfoi".md

In [8]:
import re
import time

# Connect to the database
conn, cursor = connect_database()
if not conn or not cursor:
    print("database connection failed")
    
# Initialize an empty list to collect all data
dataall = []

# Define the maximum number of retries for each SQL query
max_retries = 3

# Iterate through each SQL query in the sql_queries list
for idx, sql_query in enumerate(sql_queries, start=1):
    print(f"Executing SQL Query {idx}/{len(sql_queries)}:\n{sql_query}\n")
    
    attempt = 0
    while attempt < max_retries:
        data, error = execute_sql(cursor, sql_query)
        
        if error:
            print(f"SQL Execution Error on Query {idx}: {error}\n")
            
            # Prepare the correction prompt for OpenAI
            correction_prompt = (
                f"The following SQL query resulted in an error. Please correct it based on the error message.\n\n"
                f"Error Message: {error}\n\n"
                f"Original SQL Query:\n{sql_query}\n\n"
                f"Provide the corrected SQL query."
            )
            
            # Request OpenAI to correct the SQL query
            corrected_sql = generate_response(correction_prompt)
            
            if not corrected_sql:
                print("Failed to correct SQL query. Skipping to the next query.\n")
                break  # Exit the retry loop and proceed to the next query
            
            print(f"Corrected SQL Query {idx}:\n\n{corrected_sql}\n")
            
            # Update the sql_query with the corrected version
            sql_query = corrected_sql
            attempt += 1
            time.sleep(1)  # Brief pause before retrying
        else:
            print(f"SQL Query {idx} executed successfully.\n")
            # Append the retrieved data to dataall
            if data:
                dataall.extend(data)
                print(f"Retrieved {len(data)} records from Query {idx}.\n")
            else:
                print(f"No data returned from Query {idx}.\n")
            break  # Exit the retry loop and proceed to the next query
    
    # After max_retries, check if the last attempt resulted in an error
    if error and attempt == max_retries:
        print(f"Reached maximum retry attempts for Query {idx}. Unable to execute this query.\n")

# Close the database connection after all queries have been processed
cursor.close()
conn.close()
print("Database connection closed.")

# Optional: Display the aggregated data
print(f"Total records retrieved from all queries: {len(dataall)}")


Successfully connected to the database.
Executing SQL Query 1/5:
SELECT 
    maude."DEVICE".generic_name AS DeviceType,
    COUNT(maude."mdrfoi".mdr_report_key) AS EventCount
FROM 
    maude."mdrfoi"
JOIN 
    maude."DEVICE" ON maude."mdrfoi".mdr_report_key = maude."DEVICE".mdr_report_key
WHERE 
    maude."mdrfoi".adverse_event_flag = 'Y'
GROUP BY 
    maude."DEVICE".generic_name
ORDER BY 
    EventCount DESC
LIMIT 10;

SQL Query 1 executed successfully.

Retrieved 10 records from Query 1.

Executing SQL Query 2/5:
SELECT 
    maude."DEVICE".device_age_text AS DeviceAge,
    COUNT(maude."mdrfoi".mdr_report_key) AS EventCount
FROM 
    maude."mdrfoi"
JOIN 
    maude."DEVICE" ON maude."mdrfoi".mdr_report_key = maude."DEVICE".mdr_report_key
WHERE 
    maude."mdrfoi".adverse_event_flag = 'Y'
GROUP BY 
    maude."DEVICE".device_age_text
ORDER BY 
    EventCount DESC
LIMIT 10;

SQL Query 2 executed successfully.

Retrieved 10 records from Query 2.

Executing SQL Query 3/5:
SELECT 
    maude.

In [9]:
print(f"{sql_query_full}\n\n sql execution outcome as follow:\n {dataall}")

To address the research question and follow the execution steps, we need to generate SQL queries that cover the following aspects:

1. **Variation in Adverse Events and Device Problems by Device Type**
2. **Device Age and Adverse Events**
3. **Manufacturer Influence**
4. **Patient Demographics**
5. **Event Location and Reporting Patterns**

Given the constraints and requirements, here are the SQL queries:

### 1. Variation in Adverse Events and Device Problems by Device Type

```sql
SELECT 
    maude."DEVICE".generic_name AS DeviceType,
    COUNT(maude."mdrfoi".mdr_report_key) AS EventCount
FROM 
    maude."mdrfoi"
JOIN 
    maude."DEVICE" ON maude."mdrfoi".mdr_report_key = maude."DEVICE".mdr_report_key
WHERE 
    maude."mdrfoi".adverse_event_flag = 'Y'
GROUP BY 
    maude."DEVICE".generic_name
ORDER BY 
    EventCount DESC
LIMIT 10;
```

### 2. Device Age and Adverse Events

```sql
SELECT 
    maude."DEVICE".device_age_text AS DeviceAge,
    COUNT(maude."mdrfoi".mdr_report_key) AS Eve

In [10]:
# 9. Analyze data to validate the research question
if dataall:
    # Construct the formatted string and assign it to a new variable
    tmp = f"{sql_query_full}\n\nSQL Execution Outcome:\n{dataall}"

    analyze_data(research_question, data)

Analysis Report:
### Analysis Report: Validity and Feasibility of the Research Question

#### 1. **Research Question Overview**

**Research Question:**
"How do the rates of adverse events and device problems vary across different types of medical devices, and what factors (such as device age, manufacturer, and patient demographics) are associated with these variations?"

**Rationale:**
The research question aims to explore variations in adverse events and device problems across different medical devices, considering factors like device age, manufacturer, and patient demographics. The analysis intends to leverage the MAUDE (Manufacturer and User Facility Device Experience) database, which contains rich information on medical device adverse events.

#### 2. **Data Analysis**

**Data Provided:**
The data provided includes event counts categorized by event locations. The event locations are coded as follows:
- `null`: Unknown location
- `000`: Hospital
- `004`: Outpatient clinic
- `001`: A